# Get Data

In [ ]:
import pandas as pd

dataLamaPath = 'data/new-fix-data.csv'
dataBaruPath = 'data/tempat-wisata-di-yogyakarta-overview.csv'
dataByPlace = 'data/byPlaceData.csv'
# dataNormalizedByPlace = 'data/normalized_byPlaceData.csv'

dfOld = pd.read_csv(dataLamaPath)
dfNew = pd.read_csv(dataBaruPath)

dfByPlace = pd.read_csv(dataByPlace)
# dfNormalized = pd.read_csv(dataNormalizedByPlace)

In [ ]:
dfOld.head()

In [ ]:
dfNew.head()

In [ ]:
dfByPlace.head()

# Data Assessing

In [ ]:
# Data Lama
dfOld.columns

In [ ]:
# Data Baru
dfNew.columns

In [ ]:
dfByPlace.columns

In [ ]:
dfOld.reviews.iloc[0]

In [ ]:
dfOld.name.iloc[0]

Data yang diambil: 
- name
- address
- coordinates
- workday_timing
- closed_on
- reviews
- rating
- most_popular_times
- popular_times
- reviewer_name
- rating_review
- review_text
- published_at_date
- accessibility_enabled
- planning_enabled
- children_enabled

## Cek Informasi Kolom

## Merampingkan Data

In [ ]:
# grpByReviewer = dfOld.groupby('name').agg({
#     'address': lambda x: list(x),
#     'coordinates': lambda x: list(x),
#     'workday_timing': lambda x: list(x),
#     'closed_on': lambda x: list(x),
#     'reviews': lambda x: list(x),
#     'rating': lambda x: list(x),
#     'most_popular_times': lambda x: list(x),
#     'popular_times': lambda x: list(x),
#     'popular_times': lambda x: list(x),
#     'reviewer_name': lambda x: list(x),
#     'rating_review': lambda x: list(x),
#     'review_text': lambda x: list(x),
#     'published_at_date': lambda x: list(x),
#     'accessibility_enabled': lambda x: list(x),
#     'planning_enabled': lambda x: list(x),
#     'children_enabled': lambda x: list(x)
# }).reset_index()

# grpByReviewer.to_csv('data/byPlaceData.csv', index=False)

## Normalisasi Bentuk

Percobaan

In [ ]:
# Percobaan 1
columns_to_normalize = ['address', 'coordinates', 'workday_timing', 'closed_on',
       'reviews', 'most_popular_times', 'popular_times', 
       'accessibility_enabled', 'planning_enabled', 'children_enabled']

dfByPlace_normalized = dfByPlace.groupby('name').agg(
    {col: lambda x:', '.join(x.dropna().unique()) for col in columns_to_normalize}
).reset_index()

dfByPlace = dfByPlace.drop(columns=columns_to_normalize).drop_duplicates(subset=['name'])
dfByPlace_updated = dfByPlace.merge(dfByPlace_normalized, on='name', how='left')

dfByPlace_updated.to_csv(dataByPlace, index=False)

dataByPlace

In [ ]:
# Percobaan 2
columns_to_normalize = ['address', 'coordinates', 'workday_timing', 'closed_on',
                        'reviews', 'most_popular_times', 'popular_times', 
                        'accessibility_enabled', 'planning_enabled', 'children_enabled']

# Normalisasi setiap kolom dengan menggabungkan nilai-nilai duplikat yang ada
for col in columns_to_normalize:
    dfByPlace[col] = dfByPlace.groupby('name')[col].transform(lambda x: ', '.join(x.dropna().unique()))

# Menyimpan hasil ke file yang sama tanpa menghapus kolom atau baris
dfByPlace.to_csv('path_to_your_csv_file.csv', index=False)

In [ ]:
# Percobaan 3

# Kolom-kolom yang berbentuk list dan perlu dipisahkan ke dalam baris yang terpisah
list_columns = [
    'coordinates', 'workday_timing', 'closed_on', 'reviews',
    'most_popular_times', 'popular_times', 'accessibility_enabled',
    'planning_enabled', 'children_enabled'
]

# Fungsi untuk memisahkan elemen list ke dalam baris yang terpisah
def explode_column(dfByPlace, column):
    dfByPlace[column] = dfByPlace[column].apply(lambda x: x.split(',') if isinstance(x, str) else [])
    dfByPlace = dfByPlace.explode(column).reset_index(drop=True)
    return dfByPlace

# Memisahkan setiap kolom yang berisi list
for column in list_columns:
    dfByPlace = explode_column(dfByPlace, column)

# Menghapus duplikasi
dfByPlace = dfByPlace.drop_duplicates()

# Melakukan normalisasi data (gabung kembali jika diperlukan)
# Misalnya normalisasi pada kolom 'reviews'
dfByPlace['coordinates'] = dfByPlace.groupby(['unique_key'])['coordinates'].transform(lambda x: ','.join(x))
dfByPlace['working_timing'] = dfByPlace.groupby(['unique_key'])['working_timing'].transform(lambda x: ','.join(x))
dfByPlace['closed_on'] = dfByPlace.groupby(['unique_key'])['closed_on'].transform(lambda x: ','.join(x))
dfByPlace['reviews'] = dfByPlace.groupby(['unique_key'])['reviews'].transform(lambda x: ','.join(x))
dfByPlace['most_popular_times'] = dfByPlace.groupby(['unique_key'])['most_popular_times'].transform(lambda x: ','.join(x))
dfByPlace['popular_times'] = dfByPlace.groupby(['unique_key'])['popular_times'].transform(lambda x: ','.join(x))
dfByPlace['accessibility_enabled'] = dfByPlace.groupby(['unique_key'])['accessibility_enabled'].transform(lambda x: ','.join(x))
dfByPlace['planning_enabled'] = dfByPlace.groupby(['unique_key'])['planning_enabled'].transform(lambda x: ','.join(x))
dfByPlace['children_enabled'] = dfByPlace.groupby(['unique_key'])['children_enabled'].transform(lambda x: ','.join(x))

# Menyimpan kembali dataset yang telah dinormalisasi
output_file_path = 'data/normalized_byPlaceData.csv'
dfByPlace.to_csv(output_file_path, index=False)

print(f"Data yang telah dinormalisasi telah disimpan di {output_file_path}")

In [ ]:
# Percobaan 4

chunksize = 10000  # Ukuran chunks (dapat disesuaikan)

# Kolom yang berbentuk list dan perlu dipisahkan ke dalam baris yang terpisah
list_columns = [
    'coordinates', 'workday_timing', 'closed_on', 'reviews',
    'most_popular_times', 'popular_times', 'accessibility_enabled',
    'planning_enabled', 'children_enabled'
]

# Membuat file output kosong
output_file_path = 'data/normalized_byPlaceData.csv'
header_written = False

# Proses per chunk
for chunk in pd.read_csv('data/byPlaceData.csv', chunksize=chunksize):
    for column in list_columns:
        # Memisahkan kolom yang berisi list
        chunk[column] = chunk[column].apply(lambda x: x.split(',') if isinstance(x, str) else [])
        chunk = chunk.explode(column).reset_index(drop=True)
    
    # Menghapus duplikasi
    chunk = chunk.drop_duplicates()

    # Simpan chunk yang telah diproses ke file CSV
    if not header_written:
        chunk.to_csv(output_file_path, mode='w', index=False, header=True)
        header_written = True
    else:
        chunk.to_csv(output_file_path, mode='a', index=False, header=False)

print(f"Data yang telah dinormalisasi telah disimpan di {output_file_path}")

## Ambil Data 

### Rating 

In [ ]:
iterator = 0

def update_rating(data1, data2, iterator):
    # Loop melalui setiap baris di data1
    for idx, row in data2.iterrows():
        # Cari baris di data2 dengan nama_wisata yang sama
        matching_row = data1[data1['name'] == row['name']]
        
        if not matching_row.empty:
            print('Ada data', row['name'])
            # Jika ada kecocokan, perbarui jumlah_reviews di data1
            data1.at[idx, 'rating'] = matching_row['rating'].values[0]
            data1.at[idx, 'is_rating_updated'] = 1
            iterator = iterator + 1
        else:
            print('Tidak ada data', row['name'])
            data1.at[idx, 'is_rating_updated'] = 0
    
    # Kembalikan data1 yang sudah diperbarui
    return data1, iterator

updatedDf, iterator = update_rating(dfByPlace, dfNew, iterator)
print(iterator)
updatedDf.head()


### Jumlah Reviews 

In [ ]:
dfOld[dfOld['name'] == 'Taman Pintar Yogyakarta'].reviews.sample(1)

In [ ]:
iterator = 0

def update_reviews(data1, data2, iterator):
    # Loop melalui setiap baris di data1
    for idx, row in data2.iterrows():
        # Cari baris di data2 dengan nama_wisata yang sama
        matching_row = data1[data1['name'] == row['name']]
        
        if not matching_row.empty:
            print('Ada data', row['name'])
            # Jika ada kecocokan, perbarui jumlah_reviews di data1
            data1.at[idx, 'reviews'] = matching_row['reviews'].values[0]
            data1.at[idx, 'is_reviews_updated'] = 1
            iterator = iterator + 1
        else:
            print('Tidak ada data', row['name'])
            data1.at[idx, 'is_reviews_updated'] = 0
    
    # Kembalikan data1 yang sudah diperbarui
    return data1, iterator

updatedDf, iterator = update_reviews(dfByPlace, dfNew, iterator)
print(iterator)
updatedDf.head()


In [ ]:
dfOld[dfOld['name'] == 'Taman Pintar Yogyakarta'].reviews.sample(1)

# Data Cleaning

## Duplicated Data

## Missing Value

## Data Null

## Feature Selection